# Mental Health based conversation LLM

In [1]:
# Load Data
from datasets import load_dataset
ds = load_dataset("Amod/mental_health_counseling_conversations")

In [2]:
# Transfer data to dataFrame
import pandas as pd
df = pd.DataFrame(ds['train'])
print(df.head())

                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [3]:
# Prompt
prompt = df["Context"][0]
print("Prompt:", prompt)

Prompt: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?


# Fine-tune Model

In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
import torch
from datasets import load_dataset as hf_load_dataset

def load_dataset(file_path, tokenizer):
    dataset = hf_load_dataset("csv", data_files={"train": file_path})["train"]
    def tokenize(example):
        return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)
    dataset = dataset.map(tokenize, batched=True)
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return dataset

def load_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):

    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    model = GPT2LMHeadModel.from_pretrained(model_name)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
        logging_dir="./logs",
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

#Train the model
train(
    train_file_path="train_data.csv",
    model_name="gpt2",
    output_dir="./result",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500
)



Step,Training Loss
10,3.293200
20,3.148900
30,3.086700
40,3.103600
50,3.021300
60,3.038700
70,2.994500
80,2.955700
90,2.910300
100,2.980400


# Baseline Evaluation, Fine-tune Evaluation, and Comparison

In [30]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Your evaluation prompt (from dataset)
prompt = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"]

# Helper function to compute NLL
def compute_nll(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        return outputs.loss.item()

# Helper function to generate text
def generate_response(model, input_ids, max_length=50):
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=input_ids.shape[1] + max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ========== 1. Load Baseline GPT-2 ==========
baseline_model = GPT2LMHeadModel.from_pretrained("gpt2")
baseline_model.eval()

baseline_nll = compute_nll(baseline_model, input_ids)
baseline_response = generate_response(baseline_model, input_ids)

# ========== 2. Load Fine-tuned GPT-2 ==========
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./result")
fine_tuned_model.eval()

fine_tuned_nll = compute_nll(fine_tuned_model, input_ids)
fine_tuned_response = generate_response(fine_tuned_model, input_ids)

# ========== 3. Print Results ==========
print("\n=== Evaluation Prompt ===")
print(prompt)

print("\n--- Baseline GPT-2 Output ---")
print("Generated Response:", baseline_response)
print("Negative Log-Likelihood:", round(baseline_nll, 4))

print("\n--- Fine-Tuned GPT-2 Output ---")
print("Generated Response:", fine_tuned_response)
print("Negative Log-Likelihood:", round(fine_tuned_nll, 4))



=== Evaluation Prompt ===
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?

--- Baseline GPT-2 Output ---
Generated Response: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone? I have an answer in my head.

My first impulse was to just sit there and cry. I know I know the answers. I know I've been there and done it. I know I'm going to find a way to make this
Negative Log-Likelihood: 3.0054

--- Fine-Tuned GPT-2 Output ---
Generated Response: I'm go